In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
#import pickle
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
#from scipy.cluster.hierarchy import linkage
from sklearn.metrics import pairwise_distances
#import matplotlib.pyplot as plt
#import seaborn as sns

import settings as conf
import metadata
from utils import chunker

# Load gene associations

## Clinvar

In [3]:
clinvar_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'clinvar-gene_associations.pkl.xz')
display(clinvar_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/clinvar-gene_associations.pkl.xz'

In [4]:
clinvar_genes_associations = pd.read_pickle(clinvar_genes_associations_filename)

In [5]:
clinvar_genes_associations.shape

(4194, 5586)

In [6]:
clinvar_genes_associations.head(5)

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,"Myopia 21, autosomal dominant",Paget disease of bone 6,ZNF711-Related X-linked Mental Retardation,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
gene,,,,,,,,,,,,,,,,,,,,,
A2M,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A4GALT,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAS,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAGAB,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## S-MultiXcan

In [7]:
smultixcan_genes_associations_zscores_filename = os.path.join(conf.GENE_ASSOC_DIR, f'smultixcan-mashr-zscores.pkl.xz')
display(smultixcan_genes_associations_zscores_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan-mashr-zscores.pkl.xz'

In [8]:
smultixcan_genes_associations_zscores = pd.read_pickle(smultixcan_genes_associations_zscores_filename)

In [9]:
smultixcan_genes_associations_zscores.shape

(22515, 4091)

In [10]:
smultixcan_genes_associations_zscores.head(5)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.169468,0.102558,0.239545,0.887758,1.313448,1.472148,0.726160,1.516367,1.299771,1.068093,...,0.813014,0.275993,0.510834,0.024717,0.430951,0.824314,0.367414,1.377624,0.738444,0.298259
ENSG00000000457,1.358856,1.846875,0.139324,0.129530,0.757757,1.103979,0.612418,1.822327,2.035372,1.008058,...,1.441795,0.654791,2.545653,1.202984,0.514244,0.237223,0.414171,0.101731,1.012735,0.945167
ENSG00000000460,0.151008,1.173202,1.179426,0.571656,0.098771,0.221072,0.276415,0.461381,0.855502,0.201876,...,0.668962,0.300040,0.541782,1.033308,0.482261,0.695624,0.336480,0.083316,3.493196,0.991948
ENSG00000000938,1.302722,0.841524,1.578926,0.721340,0.139314,4.387016,0.125959,1.247123,0.215124,0.892083,...,0.126657,0.048048,1.886356,0.540496,0.127524,1.494501,0.056432,1.704863,1.351619,1.027297
ENSG00000000971,1.338813,0.262339,0.689379,1.702019,0.325859,0.063161,1.141126,0.882682,0.035533,1.810191,...,0.858497,1.675562,2.319072,1.598721,0.162958,0.005703,3.004544,0.803669,0.444266,0.165671


### Compute unique list of genes and non-empty diseases

In [11]:
clinvar_genes_to_id = pd.Index([
    metadata.GENE_NAME_TO_ID_MAP[x]
    for x in clinvar_genes_associations.index
    if x in metadata.GENE_NAME_TO_ID_MAP
])
display(len(clinvar_genes_to_id))

shared_gene_ids = clinvar_genes_to_id.intersection(smultixcan_genes_associations_zscores.index)
display(len(shared_gene_ids))

shared_gene_names = pd.Index([metadata.GENE_ID_TO_NAME_MAP[x] for x in shared_gene_ids])
display(len(shared_gene_names))

assert len(shared_gene_ids) == len(shared_gene_names)

3790

3789

3789

In [12]:
smultixcan_genes_associations_zscores = smultixcan_genes_associations_zscores.loc[shared_gene_ids]
clinvar_genes_associations = clinvar_genes_associations.loc[shared_gene_names]

In [13]:
_tmp = clinvar_genes_associations.sum()
_clinvar_diseases_to_remove = _tmp[_tmp == 0].index
display(_clinvar_diseases_to_remove.shape)

(480,)

In [14]:
# remove diseases that have no-genes associated (after selecting just shared ones above)
clinvar_genes_associations = clinvar_genes_associations.drop(columns=_clinvar_diseases_to_remove)

In [15]:
_tmp = clinvar_genes_associations.sum()
assert _tmp[_tmp == 0].shape[0] == 0

In [16]:
display(smultixcan_genes_associations_zscores.shape)
display(clinvar_genes_associations.shape)

(3789, 4091)

(3789, 5106)

# Compute associations

In [17]:
smultixcan_squared_zscores = np.power(smultixcan_genes_associations_zscores.T.fillna(0).values, 2)

In [18]:
smultixcan_squared_zscores.shape

(4091, 3789)

In [19]:
clinvar_genes_associations.shape

(3789, 5106)

In [20]:
z2_sums = np.dot(smultixcan_squared_zscores, clinvar_genes_associations.values)

In [21]:
z2_sums.shape

(4091, 5106)

In [22]:
n_clinvar_genes = clinvar_genes_associations.sum().to_frame().T
n_clinvar_genes = pd.concat([n_clinvar_genes] * z2_sums.shape[0], ignore_index=True).values

In [23]:
n_clinvar_genes.shape

(4091, 5106)

In [24]:
pheno_corrs_results = z2_sums / n_clinvar_genes

In [25]:
pheno_corrs_results.shape

(4091, 5106)

In [26]:
assoc_coef_df = pd.DataFrame(
    pheno_corrs_results,
    index=smultixcan_genes_associations_zscores.columns.copy(),
    columns=clinvar_genes_associations.columns.copy()
)

In [27]:
assert assoc_coef_df.shape == (smultixcan_genes_associations_zscores.shape[1], clinvar_genes_associations.shape[1])
display(assoc_coef_df.shape)

(4091, 5106)

In [28]:
assoc_coef_df.head()

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,Retinitis pigmentosa 58,"Myopia 21, autosomal dominant",Paget disease of bone 6,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
20096_1-Size_of_red_wine_glass_drunk_small_125ml,0.651161,0.247607,3.820126,1.075217,0.009677,0.162067,1.670074,1.670074,0.273777,0.273777,...,0.292204,1.695379,0.355525,1.101905,0.379410,1.747519,0.272920,0.006041,0.194577,0.194577
2345-Ever_had_bowel_cancer_screening,0.544953,0.055165,0.373364,0.667099,1.731603,5.829613,0.000311,0.000311,0.872317,0.872317,...,0.871863,0.772939,0.120311,0.015428,0.332303,1.911665,2.069227,0.242165,0.392449,0.392449
N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,2.352883,3.096874,0.208791,1.486795,0.166468,0.268138,2.304044,2.304044,0.007125,0.007125,...,0.020070,0.137331,0.744516,0.810886,0.222463,5.448538,0.087805,0.725028,0.155316,0.155316
100011_raw-Iron,2.905654,0.049563,11.132239,0.978171,0.428900,0.395410,0.199536,0.199536,0.638303,0.638303,...,0.000193,0.327350,0.436058,0.096868,0.173245,3.459105,0.144557,0.009105,0.015222,0.015222
5221-Index_of_best_refractometry_result_right,0.836999,0.071389,0.229481,0.340287,0.546314,2.319034,4.984327,4.984327,0.177313,0.177313,...,2.239274,1.115672,0.141618,0.012610,1.253595,0.031501,0.244870,0.001039,0.028283,0.028283


In [29]:
# testing
_pheno0 = "22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters"
_pheno1 = "Alzheimer's disease"

_genes = clinvar_genes_associations[_pheno1].sort_values(ascending=False)
display(_genes.head(10))
_genes = _genes[_genes > 0].index
display(_genes)

A2M      1
MPO      1
APP      1
NOS3     1
PLAU     1
HFE      1
FOXH1    0
FOXF1    0
FOXG1    0
FOXL2    0
Name: Alzheimer's disease, dtype: uint8

Index(['A2M', 'MPO', 'APP', 'NOS3', 'PLAU', 'HFE'], dtype='object')

In [30]:
_tmp = smultixcan_genes_associations_zscores[_pheno0].rename(index=metadata.GENE_ID_TO_NAME_MAP)
display(_tmp.head())

A2M       0.181998
A2ML1     1.054712
A4GALT    2.209362
AAAS      0.598121
AAGAB     0.363092
Name: 22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters, dtype: float64

In [31]:
_tmp.loc[_genes]

A2M     0.181998
MPO     1.749522
APP     0.022963
NOS3    0.873820
PLAU    0.178090
HFE     0.208738
Name: 22617_5223-Job_SOC_coding_Metal_working_production_and_maintenance_fitters, dtype: float64

In [32]:
_obs = np.power(_tmp.loc[_genes], 2).sum() / 6
display(_obs)
_exp = assoc_coef_df.loc[_pheno0, _pheno1]
display(_exp)
assert np.isclose(_obs, _exp), (_obs, _exp)

0.6555543699722598

0.6555543699722599

In [33]:
# testing
_pheno0 = '30040_raw-Mean_corpuscular_volume'
_pheno1 = "Alzheimer's disease"

_genes = clinvar_genes_associations[_pheno1].sort_values(ascending=False)
display(_genes.head(10))
_genes = _genes[_genes > 0].index
display(_genes)

A2M      1
MPO      1
APP      1
NOS3     1
PLAU     1
HFE      1
FOXH1    0
FOXF1    0
FOXG1    0
FOXL2    0
Name: Alzheimer's disease, dtype: uint8

Index(['A2M', 'MPO', 'APP', 'NOS3', 'PLAU', 'HFE'], dtype='object')

In [34]:
_tmp = smultixcan_genes_associations_zscores[_pheno0].rename(index=metadata.GENE_ID_TO_NAME_MAP)
display(_tmp.head())

A2M       1.370702
A2ML1     0.541736
A4GALT    6.618485
AAAS      6.178175
AAGAB     0.975214
Name: 30040_raw-Mean_corpuscular_volume, dtype: float64

In [35]:
_tmp.loc[_genes]

A2M      1.370702
MPO      2.919041
APP      0.544944
NOS3     2.223614
PLAU     1.466185
HFE     40.000000
Name: 30040_raw-Mean_corpuscular_volume, dtype: float64

In [36]:
_obs = np.power(_tmp.loc[_genes], 2).sum() / 6
display(_obs)
_exp = assoc_coef_df.loc[_pheno0, _pheno1]
display(_exp)
assert np.isclose(_obs, _exp), (_obs, _exp)

269.6317906903394

269.6317906903394

### Save

In [37]:
# sort index like smultixcan results
assoc_coef_df = assoc_coef_df.loc[smultixcan_genes_associations_zscores.columns]

In [38]:
assoc_coef_df.index.rename('phenomexcan_traits', inplace=True)

In [39]:
assoc_coef_df_filename = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan_and_clinvar-z2.pkl.xz')
display(assoc_coef_df_filename)

'/mnt/phenomexcan_base/gene_assoc/smultixcan_and_clinvar-z2.pkl.xz'

In [40]:
assoc_coef_df.to_pickle(assoc_coef_df_filename)

In [41]:
pd.Series(assoc_coef_df.values.flatten()).describe()

count    2.088865e+07
mean     1.330285e+00
std      7.561057e+00
min      0.000000e+00
25%      1.233913e-01
50%      5.334078e-01
75%      1.485715e+00
max      1.600000e+03
dtype: float64

In [42]:
assert not assoc_coef_df.isna().any().any()

In [43]:
assert not (assoc_coef_df < 0).any().any()

In [44]:
assert np.isfinite(assoc_coef_df).all().all()

In [45]:
# some testing
ukb_pheno = '20002_1111-Noncancer_illness_code_selfreported_asthma'
clinvar_pheno = 'Asthma, susceptibility to'

assert assoc_coef_df.loc[ukb_pheno, clinvar_pheno] == 28.748716710063185

### Save for publication

In [46]:
output_file = os.path.join(conf.GENE_ASSOC_DIR, 'smultixcan_and_clinvar-z2.tsv.gz')
display(output_file)

assoc_coef_df.to_csv(output_file, sep='\t', float_format='%.4e')

'/mnt/phenomexcan_base/gene_assoc/smultixcan_and_clinvar-z2.tsv.gz'

In [47]:
_tmp = pd.read_csv(output_file, sep='\t', index_col='phenomexcan_traits')

In [48]:
display(_tmp.shape)
assert _tmp.shape == assoc_coef_df.shape

(4091, 5106)

In [49]:
_tmp.head()

,Alzheimer's disease,Alpha-2-macroglobulin deficiency,"Otitis media, susceptibility to",p phenotype,Glucocorticoid deficiency with achalasia,Keratosis palmoplantaris papulosa,"Charcot-Marie-Tooth disease, type 2N","Epileptic encephalopathy, early infantile, 29",Combined oxidative phosphorylation deficiency 8,"Leukoencephalopathy, progressive, with ovarian failure",...,Retinitis pigmentosa 58,"Myopia 21, autosomal dominant",Paget disease of bone 6,Seborrhea-like dermatitis with psoriasiform elements,PEHO syndrome,Oocyte maturation defect 1,OOCYTE MATURATION DEFECT 6,OOCYTE MATURATION DEFECT 3,Acromelic frontonasal dysostosis,"NEURODEVELOPMENTAL DISORDER WITH MOVEMENT ABNORMALITIES, ABNORMAL GAIT, AND AUTISTIC FEATURES"
phenomexcan_traits,,,,,,,,,,,,,,,,,,,,,
20096_1-Size_of_red_wine_glass_drunk_small_125ml,0.65116,0.247610,3.82010,1.07520,0.009677,0.16207,1.670100,1.670100,0.273780,0.273780,...,0.292200,1.69540,0.35553,1.101900,0.37941,1.747500,0.272920,0.006041,0.194580,0.194580
2345-Ever_had_bowel_cancer_screening,0.54495,0.055165,0.37336,0.66710,1.731600,5.82960,0.000311,0.000311,0.872320,0.872320,...,0.871860,0.77294,0.12031,0.015428,0.33230,1.911700,2.069200,0.242160,0.392450,0.392450
N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,2.35290,3.096900,0.20879,1.48680,0.166470,0.26814,2.304000,2.304000,0.007125,0.007125,...,0.020070,0.13733,0.74452,0.810890,0.22246,5.448500,0.087805,0.725030,0.155320,0.155320
100011_raw-Iron,2.90570,0.049563,11.13200,0.97817,0.428900,0.39541,0.199540,0.199540,0.638300,0.638300,...,0.000193,0.32735,0.43606,0.096868,0.17324,3.459100,0.144560,0.009105,0.015222,0.015222
5221-Index_of_best_refractometry_result_right,0.83700,0.071389,0.22948,0.34029,0.546310,2.31900,4.984300,4.984300,0.177310,0.177310,...,2.239300,1.11570,0.14162,0.012610,1.25360,0.031501,0.244870,0.001039,0.028283,0.028283


In [50]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [51]:
assert not _tmp.isna().any().any()

In [52]:
assert ((_tmp >= 0) & (_tmp <= 1600)).all().all()

In [53]:
assert np.allclose(_tmp.values, assoc_coef_df.values, atol=1e-15, rtol=1e-4)